In [1]:
import wandb

wandb.init(project="bmi-LSTM", entity="jmjung1997")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jmjung1997. Use `wandb login --relogin` to force relogin


In [2]:
wandb.config = {
  "learning_rate": 0.001,
  "epochs": 100,
  "batch_size": 128
}

In [3]:
# wandb.log({"loss": loss})


In [4]:
import os
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wandb
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split  

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:
!pip install wandb

In [6]:
# wandb login

In [7]:
data=pd.read_excel('./modeling_data/Data_Timeseries.xlsx')
print(data.dtypes)

Date              int64
ID               object
height          float64
weight          float64
step_count      float64
burn_calorie    float64
eat_calorie     float64
sleep           float64
bmi             float64
bmi_target        int64
dtype: object


In [8]:
y_origin=data['bmi_target']

In [9]:
y_origin

0        3
1        3
2        3
3        3
4        3
        ..
27043    4
27044    4
27045    4
27046    4
27047    4
Name: bmi_target, Length: 27048, dtype: int64

In [10]:
y=pd.get_dummies(data['bmi_target'])

In [11]:
# data['Date'] = pd.to_datetime(data['Date'])

In [12]:
# data.set_index('Date', inplace=True)
X=data.iloc[:,2:9]
y=y.iloc[0:84]
print(X)
print(y)

       height  weight  step_count  burn_calorie  eat_calorie  sleep        bmi
0       148.0    45.0        72.0           0.0          0.0    0.0  20.544193
1       148.0    45.0        72.0           0.0          0.0    0.0  20.544193
2       148.0    45.0        72.0           0.0          0.0    0.0  20.544193
3       148.0    45.0        72.0           0.0          0.0    0.0  20.544193
4       148.0    45.0        72.0           0.0          0.0    0.0  20.544193
...       ...     ...         ...           ...          ...    ...        ...
27043   153.0    68.0         0.0           0.0          0.0    0.0  29.048656
27044   153.0    68.0         0.0           0.0          0.0    0.0  29.048656
27045   153.0    68.0         0.0           0.0          0.0    0.0  29.048656
27046   153.0    68.0         0.0           0.0          0.0    0.0  29.048656
27047   153.0    68.0         0.0           0.0          0.0    0.0  29.048656

[27048 rows x 7 columns]
    1  2  3  4
0   0  0  1

In [13]:
type(y)

pandas.core.frame.DataFrame

In [14]:
ms = MinMaxScaler()
ss = StandardScaler()
X_ss = ss.fit_transform(X)
# y_ms = ms.fit_transform(y) 
y_ms=y

X_train = X_ss[:57, :]
X_test = X_ss[57:84, :]

y_train = y_ms[:57]
y_test = y_ms[57:] 

y_train=y_train.to_numpy()
y_test=y_test.to_numpy()
print("Training Shape", X_train.shape, y_train.shape)
print("Testing Shape", X_test.shape, y_test.shape) 

Training Shape (57, 7) (57, 4)
Testing Shape (27, 7) (27, 4)


In [15]:
X_train_tensors = Variable(torch.Tensor(X_train))
X_test_tensors = Variable(torch.Tensor(X_test))

In [16]:
type(y_train)

numpy.ndarray

In [17]:
y_train_tensors = Variable(torch.Tensor(y_train))
y_test_tensors = Variable(torch.Tensor(y_test))

X_train_tensors_f = torch.reshape(X_train_tensors,   (X_train_tensors.shape[0], 1, X_train_tensors.shape[1]))
X_test_tensors_f = torch.reshape(X_test_tensors,  (X_test_tensors.shape[0], 1, X_test_tensors.shape[1])) 




print("Training Shape", X_train_tensors_f.shape, y_train_tensors.shape)
print("Testing Shape", X_test_tensors_f.shape, y_test_tensors.shape) 

Training Shape torch.Size([57, 1, 7]) torch.Size([57, 4])
Testing Shape torch.Size([27, 1, 7]) torch.Size([27, 4])


In [18]:
wandb.init()

In [19]:
class LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM, self).__init__()
        self.num_classes = num_classes 
        self.num_layers = num_layers 
        self.input_size = input_size 
        self.hidden_size = hidden_size 
        self.seq_length = seq_length 

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) 
        self.fc_1 =  nn.Linear(hidden_size, 128) 
        self.fc = nn.Linear(128, num_classes) 

        self.relu = nn.ReLU()
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) 
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) 
        
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) 
        hn = hn.view(-1, self.hidden_size) 
        out = self.relu(hn)
        out = self.fc_1(out) 
        out = self.relu(out) 
        out = self.fc(out)
        return out

In [20]:
num_epochs = 1000 
learning_rate = 0.0001 

input_size = 7
hidden_size = 2 
num_layers = 1

num_classes = 4
model = LSTM(num_classes, input_size, hidden_size, num_layers, X_train_tensors_f.shape[1]) 

criterion = torch.nn.MSELoss()    
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [21]:
for epoch in range(num_epochs):
    outputs = model.forward(X_train_tensors_f)

    optimizer.zero_grad()  
    loss = criterion(outputs, y_train_tensors) 
    loss.backward() 
    optimizer.step() 
    wandb.log({"loss":loss})
    if epoch % 100 == 0:
        print("Epoch: %d,loss: %1.5f" % (epoch, loss.item())) 
        

Epoch: 0,loss: 0.22035
Epoch: 100,loss: 0.12144
Epoch: 200,loss: 0.06033
Epoch: 300,loss: 0.02604
Epoch: 400,loss: 0.00940
Epoch: 500,loss: 0.00269
Epoch: 600,loss: 0.00060
Epoch: 700,loss: 0.00010
Epoch: 800,loss: 0.00001
Epoch: 900,loss: 0.00000


In [22]:
df_y_ms=y
df_y_ms=df_y_ms.to_numpy()

In [23]:
df_x_ss = ss.transform(data.iloc[:84, 2:9]) 

In [24]:
#df_y_ms = ms.transform(y.iloc[:57]) 
df_x_ss = Variable(torch.Tensor(df_x_ss)) 
df_y_ms = Variable(torch.Tensor(df_y_ms))
df_x_ss = torch.reshape(df_x_ss, (df_x_ss.shape[0], 1, df_x_ss.shape[1])) 

In [25]:
train_predict = model(df_x_ss)
predicted = train_predict.data.numpy() 
label_y = df_y_ms.data.numpy()

In [26]:
predicted

array([[5.9604645e-08, 5.8114529e-07, 9.9929005e-01, 9.6857548e-08],
       [5.9604645e-08, 5.8114529e-07, 9.9929005e-01, 9.6857548e-08],
       [5.9604645e-08, 5.8114529e-07, 9.9929005e-01, 9.6857548e-08],
       [5.9604645e-08, 5.8114529e-07, 9.9929005e-01, 9.6857548e-08],
       [5.9604645e-08, 5.8114529e-07, 9.9929005e-01, 9.6857548e-08],
       [5.9604645e-08, 5.8114529e-07, 9.9929005e-01, 9.6857548e-08],
       [5.9604645e-08, 5.8114529e-07, 9.9929005e-01, 9.6857548e-08],
       [5.9604645e-08, 5.8114529e-07, 9.9929005e-01, 9.6857548e-08],
       [5.9604645e-08, 5.8114529e-07, 9.9929005e-01, 9.6857548e-08],
       [5.9604645e-08, 5.8114529e-07, 9.9929005e-01, 9.6857548e-08],
       [5.9604645e-08, 5.8114529e-07, 9.9929005e-01, 9.6857548e-08],
       [5.9604645e-08, 5.8114529e-07, 9.9929005e-01, 9.6857548e-08],
       [5.9604645e-08, 5.8114529e-07, 9.9929005e-01, 9.6857548e-08],
       [5.9604645e-08, 5.8114529e-07, 9.9929005e-01, 9.6857548e-08],
       [5.9604645e-08, 5.8114529e-

In [27]:
predicted[0]

array([5.9604645e-08, 5.8114529e-07, 9.9929005e-01, 9.6857548e-08],
      dtype=float32)

In [31]:
count=0
for i in range(len(y_test)):
    for j in range(1,4):
        max=predicted[i][0]
        maxIndex=0
        if(predicted[i][j]>max):
            max=predicted[i][j]
            maxIndex=j
    print(predicted[i])        
    print(i, " : " ,"maxIndx",maxIndex,"y_origin[i]",y_origin[i]-1)
    if y_origin[i]==maxIndex:
        print(i, " : " ,"maxIndx",maxIndex,"y_origin[i]",y_origin[i]-1)
        count=count+1

print(count)
print(len(y_test))
print(count/len(y_test))

[5.9604645e-08 5.8114529e-07 9.9929005e-01 9.6857548e-08]
0  :  maxIndx 3 y_origin[i] 2
0  :  maxIndx 3 y_origin[i] 2
[5.9604645e-08 5.8114529e-07 9.9929005e-01 9.6857548e-08]
1  :  maxIndx 3 y_origin[i] 2
1  :  maxIndx 3 y_origin[i] 2
[5.9604645e-08 5.8114529e-07 9.9929005e-01 9.6857548e-08]
2  :  maxIndx 3 y_origin[i] 2
2  :  maxIndx 3 y_origin[i] 2
[5.9604645e-08 5.8114529e-07 9.9929005e-01 9.6857548e-08]
3  :  maxIndx 3 y_origin[i] 2
3  :  maxIndx 3 y_origin[i] 2
[5.9604645e-08 5.8114529e-07 9.9929005e-01 9.6857548e-08]
4  :  maxIndx 3 y_origin[i] 2
4  :  maxIndx 3 y_origin[i] 2
[5.9604645e-08 5.8114529e-07 9.9929005e-01 9.6857548e-08]
5  :  maxIndx 3 y_origin[i] 2
5  :  maxIndx 3 y_origin[i] 2
[5.9604645e-08 5.8114529e-07 9.9929005e-01 9.6857548e-08]
6  :  maxIndx 3 y_origin[i] 2
6  :  maxIndx 3 y_origin[i] 2
[5.9604645e-08 5.8114529e-07 9.9929005e-01 9.6857548e-08]
7  :  maxIndx 3 y_origin[i] 2
7  :  maxIndx 3 y_origin[i] 2
[5.9604645e-08 5.8114529e-07 9.9929005e-01 9.6857548e-08

In [29]:
predicted

array([[5.9604645e-08, 5.8114529e-07, 9.9929005e-01, 9.6857548e-08],
       [5.9604645e-08, 5.8114529e-07, 9.9929005e-01, 9.6857548e-08],
       [5.9604645e-08, 5.8114529e-07, 9.9929005e-01, 9.6857548e-08],
       [5.9604645e-08, 5.8114529e-07, 9.9929005e-01, 9.6857548e-08],
       [5.9604645e-08, 5.8114529e-07, 9.9929005e-01, 9.6857548e-08],
       [5.9604645e-08, 5.8114529e-07, 9.9929005e-01, 9.6857548e-08],
       [5.9604645e-08, 5.8114529e-07, 9.9929005e-01, 9.6857548e-08],
       [5.9604645e-08, 5.8114529e-07, 9.9929005e-01, 9.6857548e-08],
       [5.9604645e-08, 5.8114529e-07, 9.9929005e-01, 9.6857548e-08],
       [5.9604645e-08, 5.8114529e-07, 9.9929005e-01, 9.6857548e-08],
       [5.9604645e-08, 5.8114529e-07, 9.9929005e-01, 9.6857548e-08],
       [5.9604645e-08, 5.8114529e-07, 9.9929005e-01, 9.6857548e-08],
       [5.9604645e-08, 5.8114529e-07, 9.9929005e-01, 9.6857548e-08],
       [5.9604645e-08, 5.8114529e-07, 9.9929005e-01, 9.6857548e-08],
       [5.9604645e-08, 5.8114529e-

In [30]:
predicted= ms.inverse_transform(predicted) 
label_y = ms.inverse_transform(label_y)
plt.figure(figsize=(10,6)) 
plt.axvline(x=200, c='r', linestyle='--') 

plt.plot(label_y, label='Actual Data') 
plt.plot(predicted, label='Predicted Data') 
plt.title('Time-Series Prediction')
plt.legend()
plt.show()

NotFittedError: This MinMaxScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
len(predicted)